In [34]:
import pandas as pd

In [35]:
from pathlib import Path

import sequence_sensei
import numpy as np

In [3]:
import tqdm

In [4]:
import axelrod as axl

In [6]:
import keras

In [7]:
axl.__version__

'4.2.0'

In [8]:
axl.Action(1)

C

**Outputs**

In [9]:
dfs = []
for file in Path("best_responses/").glob("*.csv"):
    dfs.append(pd.read_csv(file, index_col=0,))

In [10]:
df = pd.concat(dfs).reset_index(drop=True)

In [11]:
df = df.drop(columns=['index'])

In [12]:
len(df)

5760

**Inputs**

In [13]:
import axelrod as axl

In [14]:
strategies = {strategy().name: strategy() for strategy in axl.basic_strategies}

In [15]:
names = [s for s in strategies]

In [16]:
names

['Alternator',
 'Anti Tit For Tat',
 'Bully',
 'Cooperator',
 'Cycler DC',
 'Defector',
 'Grudger',
 'Suspicious Tit For Tat',
 'Tit For Tat',
 'Win-Shift Lose-Stay',
 'Win-Stay Lose-Shift']

In [17]:
df = df[df['opponent'].isin(names)]

In [18]:
opponents_moves = []
for i, opponent in enumerate(tqdm.tqdm_notebook(df['opponent'])):
    
    majority = 'Majority'
    if majority in opponent:
        name = opponent.split(majority)[0] + 'Majority:' + opponent.split(majority)[-1]
        if name[-1] == ":":
            name = name[:-1]
        opponent = strategies[name]
    
    else:
        opponent = strategies[opponent.split(':')[0]]
    
    sequence = sequence_sensei.get_sequence_str(df.iloc[i].values[-205:])
    cycler = axl.Cycler(sequence)

    match = axl.Match([opponent, cycler], turns=205)
    _ = match.play()
    
    moves_opp, moves_cycler = zip(*match.result)
    
    assert sequence_sensei.get_sequence_str(moves_cycler) == sequence
    
    opponents_moves.append((opponent,) + moves_opp)

**Transform Data for RNN**

In [19]:
sequences = [[moves[0]] + [move.value for move in moves[1:]] for moves in opponents_moves]

In [20]:
data = pd.DataFrame(sequences)

In [21]:
data.columns = df.columns

In [22]:
data.to_csv('data/basic_sequences.csv'), df.to_csv('data/basic_targets.csv')

(None, None)

**Transform the data**

In [23]:
sequences = pd.read_csv('data/basic_sequences.csv', index_col=0)
targets = pd.read_csv('data/basic_targets.csv', index_col=0)

In [24]:
inputs = sequences.replace({0: 2})

In [25]:
inputs = inputs.drop(columns=["opponent", "gene_204"]).values
outputs = targets.drop(columns=["opponent", "gene_0"]).values

In [26]:
max_length = len(inputs[0])

In [27]:
prep_X_train = []
prep_y_train = []

for i, sequence in enumerate(inputs):
    assert len(sequence) == max_length
    for histories in range(1, max_length + 1):
        prep_X_train.append(sequence[:histories])
        prep_y_train.append(outputs[i][histories - 1])        

In [28]:
padded_X_train = keras.preprocessing.sequence.pad_sequences(prep_X_train, maxlen=max_length, padding='post') 

In [29]:
padded_X = pd.DataFrame(padded_X_train, columns=[i for i in range(max_length)])

In [30]:
padded_X.to_csv('data/basic_padded_inputs_classification.csv')

In [31]:
y = pd.DataFrame(prep_y_train, columns=['target'])

In [32]:
y.to_csv('data/basic_classification_output.csv')

In [33]:
len(y), len(padded_X)

(17136, 17136)